Modelar o ciclo de cogeração ilustrado abaixo. Permitir que o processo varie de 
0% (desligado) até 100% (completamente ligado).

![Ciclo](img/ciclo.jpeg "Ciclo")

In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('data.xlsx')
display(df)

,Temperatura [ºC],Pressão [kPa],Vazão [kg/s],Título [-]
0,485,6495,27.90,NaN
1,NaN,900,NaN,NaN
2,NaN,250,NaN,NaN
3,51,Saturação,NaN,NaN
4,51,Saturação,NaN,0.0
5,NaN,250,NaN,NaN
6,110,250,NaN,NaN
7,NaN,6495,27.90,NaN
8,110,250,NaN,NaN
9,NaN,900,NaN,NaN


In [3]:
from CoolProp.CoolProp import PropsSI as prop

In [4]:
fluido = 'Water'

Inserindo temperatura de saturação na tabela e convertendo para K

In [21]:
T, c = [], 0

for i in df['Temperatura [ºC]']:
    c = c + 1
    try:
        if i == 'Saturação':
            T.insert(c-1, round(prop('T', 'P', df['Pressão [kPa]'][c-1], 'Q', 0, fluido), 2) )
        else:
            T.insert(c-1, i + 273.15)
    except:
        print(f'Sem informações de pressão no estado {c}')
    

Inserindo pressão de saturação na tabela 

In [22]:
P, c = [], 0
for i in df['Pressão [kPa]']:
    c = c + 1
    try:
        if i == 'Saturação':
            P.insert(c-1, round(prop('P', 'T', T[c-1], 'Q', 0, fluido),2) )
        else:
            P.insert(c-1, i)
    except:
        print(f'Sem informações de temperatura no estado {c}')

In [7]:
from CoolProp.CoolProp import State as st
from numpy import arange, absolute

In [8]:
V = df['Vazão [kg/s]']
x = df['Título [-]']

In [23]:
st1 = st(fluido, {'P': P[0], 'T': T[0] })
st4 = st(fluido, {'P': P[3], 'T': T[3] })
st5 = st(fluido, {'P': P[4], 'T': T[4] })
st7 = st(fluido, {'P': P[6], 'T': T[6] })
st9 = st(fluido, {'P': P[8], 'T': T[8]})
st11 = st(fluido, {'P': P[10], 'T': T[10]})
st12 = st(fluido, {'P': P[11], 'T': T[11]})

_Eficiência isentrópica_ <br/>
Turbina: $$ \eta_{turb} = \frac{h_{3} - h_{4}}{h_{3} - h_{4s}} $$
Bomba: $$ \eta_{pump} = \frac{h_{2s} - h_{1}}{h_{2} - h_{1}} $$

## TESTE ##

In [17]:
st2 = st(fluido, {'P': P[0], 'T': T[0] })
st3 = st(fluido, {'P': P[3], 'T': T[3] })
st6 = st(fluido, {'P': P[4], 'T': T[4] })
st8 = st(fluido, {'P': P[6], 'T': T[6] })
st10 = st(fluido, {'P': P[8], 'T': T[8]})
st11 = st(fluido, {'P': P[10], 'T': T[10]})
st12 = st(fluido, {'P': P[11], 'T': T[11]})

## FIM ##

In [18]:
estds = st1, st2, st3, st4, st5, st6, st7, st8, st9, st10, st11, st12

temperatura, pressao, entalpia, entropia, titulo, estados = [], [], [], [], [], arange(1, len(estds)+1, 1)

for i in estds: 
    temperatura.append(i.T)
    pressao.append(i.p)
    entalpia.append(i.h)
    entropia.append(i.s)
    if absolute(i.Q) == 1:
        titulo.append('Vap. Sat.')
    elif i.Q == 0:
        titulo.append('Liq. Sat.')
    else:
        titulo.append(i.Q)

In [19]:
data_st = {
    'Estado': estados,
    'Temperatura [ºC]': temperatura,
    'Pressão [kPa]': pressao,
    'Entalpia [kJ/kg]': entalpia,
    'Entropia [kJ/kg/K]': entropia,
    'Titulo [-]': titulo
}

df_st = pd.DataFrame(data_st).style.hide(axis='index').format(precision=2)

In [20]:
display(df_st)

Estado,Temperatura [ºC],Pressão [kPa],Entalpia [kJ/kg],Entropia [kJ/kg/K],Titulo [-]
1,758.15,6495.00,3381.20,6.79,Vap. Sat.
2,758.15,6495.00,3381.20,6.79,Vap. Sat.
3,324.15,12978.07,224.65,0.71,Vap. Sat.
4,324.15,12978.07,224.65,0.71,Vap. Sat.
5,324.15,12978.07,224.65,0.71,Vap. Sat.
6,324.15,12978.07,224.65,0.71,Vap. Sat.
7,383.15,250.00,461.49,1.42,Vap. Sat.
8,383.15,250.00,461.49,1.42,Vap. Sat.
9,383.15,250.00,461.49,1.42,Vap. Sat.
10,383.15,250.00,461.49,1.42,Vap. Sat.


$$ \omega_{pump, in} = h_2 - h_1 $$
$$ \omega_{turb, out} = h_3 - h_4 $$
$$ q_{in} = h_3 - h_2 $$
$$ q_{out} = h_4 - h_1 $$
$$\eta_{th} = 1 - \frac{q_{out}}{q_{in}}$$

Fazer a modelagem com condensador para que sua temperatura de condensação 
varie conforme a vazão de condensado. Mostrar essa variação.

![Condensador](img/Condensador.jpeg "Condensador")

Desenvolver a análise de primeira lei para o esquema de caldeira abaixo. 
Considerar que este sistema está inserido dentro do ciclo de potência já desenvolvido 
(Fig. 1).


![Caldeira](img/Caldeira.jpeg "Caldeira")